In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import math


In [2]:
#data_url = "https://panamadata.blob.core.windows.net/icij/"
data_url = "../offshore_leaks_csvs-20170104/"

## Look into Addresses table

In [3]:
addresses = pd.read_csv(data_url + "Addresses.csv", dtype={'note':str})
addresses.set_index(['node_id'], inplace=True)
addresses.head()

,address,icij_id,valid_until,country_codes,countries,sourceID,note
node_id,,,,,,,
14000001,-\t27 ROSEWOOD DRIVE #16-19 SINGAPORE 737920,6991059DFFB057DF310B9BF31CC4A0E6,The Panama Papers data is current through 2015,SGP,Singapore,Panama Papers,NaN
14000002,"""Almaly Village"" v.5, Almaty Kazakhstan",C65A269424AA1A8BBAC416FA30150938,The Panama Papers data is current through 2015,KAZ,Kazakhstan,Panama Papers,NaN
14000003,"""Cantonia"" South Road St Georges Hill Weybridg...",240EE44DFB70AF775E6CD02AF8CB889B,The Panama Papers data is current through 2015,GBR,United Kingdom,Panama Papers,NaN
14000004,"""CAY-OS"" NEW ROAD; ST.SAMPSON; GUERNSEY; CHANN...",340C6D3D2FC60AE58C53BB3618129C43,The Panama Papers data is current through 2015,GGY,Guernsey,Panama Papers,NaN
14000005,"""Chirag"" Plot No 652; Mwamba Road; Kizingo; Mo...",6DBCD74F5AAB63EABC166885CE8CD7CD,The Panama Papers data is current through 2015,KEN,Kenya,Panama Papers,NaN


In [4]:
addresses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151605 entries, 14000001 to 28000023
Data columns (total 7 columns):
address          151604 non-null object
icij_id          93982 non-null object
valid_until      151605 non-null object
country_codes    150714 non-null object
countries        150714 non-null object
sourceID         151605 non-null object
note             23 non-null object
dtypes: object(7)
memory usage: 9.3+ MB


#### How may sources are there?

In [5]:
addresses['sourceID'].unique()

array(['Panama Papers', 'Offshore Leaks', 'Bahamas Leaks'], dtype=object)

#### No need to have both 'countries and country_codes columns

In [6]:
addresses = addresses.drop(['countries'], axis=1)

In [7]:
addresses['note'].unique()

array([nan, 'Record manually added from leaked documents'], dtype=object)

#### Convert 'note' field into a binary field indicating whether it's manual entry

In [8]:
addresses['note'] = addresses['note'].fillna("");
addresses['manual_entry'] = [0 if note == "" else 1 for note in addresses['note'] ]
addresses.drop(['note'], axis=1, inplace=True)
addresses.head()

,address,icij_id,valid_until,country_codes,sourceID,manual_entry
node_id,,,,,,
14000001,-\t27 ROSEWOOD DRIVE #16-19 SINGAPORE 737920,6991059DFFB057DF310B9BF31CC4A0E6,The Panama Papers data is current through 2015,SGP,Panama Papers,0
14000002,"""Almaly Village"" v.5, Almaty Kazakhstan",C65A269424AA1A8BBAC416FA30150938,The Panama Papers data is current through 2015,KAZ,Panama Papers,0
14000003,"""Cantonia"" South Road St Georges Hill Weybridg...",240EE44DFB70AF775E6CD02AF8CB889B,The Panama Papers data is current through 2015,GBR,Panama Papers,0
14000004,"""CAY-OS"" NEW ROAD; ST.SAMPSON; GUERNSEY; CHANN...",340C6D3D2FC60AE58C53BB3618129C43,The Panama Papers data is current through 2015,GGY,Panama Papers,0
14000005,"""Chirag"" Plot No 652; Mwamba Road; Kizingo; Mo...",6DBCD74F5AAB63EABC166885CE8CD7CD,The Panama Papers data is current through 2015,KEN,Panama Papers,0


## Now look at the Entities table

In [9]:
entities = pd.read_csv(data_url + "Entities.csv", low_memory=False)
entities.head()

,name,original_name,former_name,jurisdiction,jurisdiction_description,company_type,address,internal_id,incorporation_date,inactivation_date,...,dorm_date,status,service_provider,ibcRUC,country_codes,countries,note,valid_until,node_id,sourceID
0,"TIANSHENG INDUSTRY AND TRADING CO., LTD.","TIANSHENG INDUSTRY AND TRADING CO., LTD.",NaN,SAM,Samoa,NaN,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1001256.0,23-MAR-2006,18-FEB-2013,...,NaN,Defaulted,Mossack Fonseca,25221,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000001,Panama Papers
1,"NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.","NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.",NaN,SAM,Samoa,NaN,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1001263.0,27-MAR-2006,27-FEB-2014,...,NaN,Defaulted,Mossack Fonseca,25249,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000002,Panama Papers
2,"HOTFOCUS CO., LTD.","HOTFOCUS CO., LTD.",NaN,SAM,Samoa,NaN,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1000896.0,10-JAN-2006,15-FEB-2012,...,NaN,Defaulted,Mossack Fonseca,24138,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000003,Panama Papers
3,"SKY-BLUE GIFTS & TOYS CO., LTD.","SKY-BLUE GIFTS & TOYS CO., LTD.",NaN,SAM,Samoa,NaN,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1000914.0,06-JAN-2006,16-FEB-2009,...,NaN,Defaulted,Mossack Fonseca,24012,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000004,Panama Papers
4,FORTUNEMAKER INVESTMENTS CORPORATION,FORTUNEMAKER INVESTMENTS CORPORATION,NaN,SAM,Samoa,NaN,LOYAL PORT LIMITED 8/F; CRAWFORD TOWER 99 JERV...,1001266.0,19-APR-2006,15-MAY-2009,...,NaN,Changed agent,Mossack Fonseca,R25638,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000005,Panama Papers


In [10]:
entities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495038 entries, 0 to 495037
Data columns (total 21 columns):
name                        495034 non-null object
original_name               389522 non-null object
former_name                 6726 non-null object
jurisdiction                495038 non-null object
jurisdiction_description    495038 non-null object
company_type                103227 non-null object
address                     299319 non-null object
internal_id                 389522 non-null float64
incorporation_date          485607 non-null object
inactivation_date           144760 non-null object
struck_off_date             272240 non-null object
dorm_date                   20207 non-null object
status                      310106 non-null object
service_provider            319150 non-null object
ibcRUC                      431263 non-null object
country_codes               318360 non-null object
countries                   318360 non-null object
note                    

In [11]:
entities["sourceID"].unique()

array(['Panama Papers', 'Offshore Leaks', 'Bahamas Leaks'], dtype=object)

In [12]:
entities["company_type"].unique()

array([nan, 'Standard International Company',
       'BVI Sundry Entities (one off transactions)',
       'Business Company Limited by Shares',
       'Standard Company under IBC Act',
       'The GRA group of Companies for express Co.', 'Sundry Account',
       'BVI Trust', 'Domestic Company', 'Client Sundry Account',
       'Mutual Fund Manager', 'Nominee Only Entity',
       'Mutual Fund Administrator', 'Professional Mutual Fund',
       'Partnership', 'Mutual Fund', 'Limited Liability Company',
       'Holding Company', 'Singapore Trust', 'Cook Islands Trust',
       'Private Company Limited By Shares', 'Collective Investment Scheme',
       'Private Trust Company', 'Cook Islands Asset Protection Trust',
       'Cook Islands Asset Protection Trust - 3520A', 'BVI Share Trust',
       'Seminar / Workshop', 'Liquidator Licence', 'SHIP REGISTRATIONS',
       'Turks', 'Trust', 'Hong Kong Trust', 'Audit Licence',
       'Nevis - Limited Liability Company', 'Bahamas IBC',
       'Belize I

In [13]:
entities["service_provider"].unique()

array(['Mossack Fonseca', 'Portcullis Trustnet',
       'Commonwealth Trust Limited', nan], dtype=object)

In [14]:
entities["status"].unique()

array(['Defaulted', 'Changed agent', 'Active', 'Inactivated',
       'Relocated in new jurisdiction', 'Dissolved shelf company',
       'Dissolved', 'Bad debt account', 'Discontinued',
       'Resigned as agent', 'Trash company', 'Shelf company',
       'Shelf company not possible to sell', 'In transition',
       'Change in administration pending', 'In liquidation', nan,
       'Resigned as agent of shelf company',
       'Struck / Defunct / Deregistered', 'In Liquidation',
       'Company liquidated', 'Transferred Out', 'Shelf', 'Dead',
       'Liquidated', 'Redomiciled',
       'Not To Be Renewed / In Deregistration', 'Vesting', 'Inactive',
       'In Formation', 'Un Registered', 'Transferred OUT',
       'Liquidation in Progress', 'Discontinuance',
       'Client Sundry Account', 'We resigned as registered agent',
       'Sundry Account', 'Not Renewed by Client'], dtype=object)

In [15]:
entities.columns

Index(['name', 'original_name', 'former_name', 'jurisdiction',
       'jurisdiction_description', 'company_type', 'address', 'internal_id',
       'incorporation_date', 'inactivation_date', 'struck_off_date',
       'dorm_date', 'status', 'service_provider', 'ibcRUC', 'country_codes',
       'countries', 'note', 'valid_until', 'node_id', 'sourceID'],
      dtype='object')

### As we can see, most of the features in this dataset are categorical. Let's pick features what we think can be relevant in clustering entities. Use 'incorporation date' and ignore other dates. We can come back to them later. Also we can add two more features indicating the day of the week and month the company is incorporated.

In [16]:
features = ['name', 'jurisdiction', 'company_type','incorporation_date', 'status', 'service_provider', 'country_codes',
       'note', 'valid_until', 'sourceID']
print(features)

['name', 'jurisdiction', 'company_type', 'incorporation_date', 'status', 'service_provider', 'country_codes', 'note', 'valid_until', 'sourceID']


In [17]:
entities = entities[features]

entities['incorporation_date'] = pd.to_datetime(entities['incorporation_date'])
entities['incorporation_month'] = entities['incorporation_date'].dt.month
entities['incorporation_dayofweek'] = entities['incorporation_date'].dt.dayofweek

#remove instances where name or incorporation date is not available.
entities.dropna(axis=0, subset=['name', 'incorporation_date'], inplace=True)
entities.head()

,name,jurisdiction,company_type,incorporation_date,status,service_provider,country_codes,note,valid_until,sourceID,incorporation_month,incorporation_dayofweek
0,"TIANSHENG INDUSTRY AND TRADING CO., LTD.",SAM,NaN,2006-03-23,Defaulted,Mossack Fonseca,HKG,NaN,The Panama Papers data is current through 2015,Panama Papers,3.0,3.0
1,"NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.",SAM,NaN,2006-03-27,Defaulted,Mossack Fonseca,HKG,NaN,The Panama Papers data is current through 2015,Panama Papers,3.0,0.0
2,"HOTFOCUS CO., LTD.",SAM,NaN,2006-01-10,Defaulted,Mossack Fonseca,HKG,NaN,The Panama Papers data is current through 2015,Panama Papers,1.0,1.0
3,"SKY-BLUE GIFTS & TOYS CO., LTD.",SAM,NaN,2006-01-06,Defaulted,Mossack Fonseca,HKG,NaN,The Panama Papers data is current through 2015,Panama Papers,1.0,4.0
4,FORTUNEMAKER INVESTMENTS CORPORATION,SAM,NaN,2006-04-19,Changed agent,Mossack Fonseca,HKG,NaN,The Panama Papers data is current through 2015,Panama Papers,4.0,2.0


In [18]:
X = entities.drop(['name', 'incorporation_date'], axis=1)
X.head()

,jurisdiction,company_type,status,service_provider,country_codes,note,valid_until,sourceID,incorporation_month,incorporation_dayofweek
0,SAM,NaN,Defaulted,Mossack Fonseca,HKG,NaN,The Panama Papers data is current through 2015,Panama Papers,3.0,3.0
1,SAM,NaN,Defaulted,Mossack Fonseca,HKG,NaN,The Panama Papers data is current through 2015,Panama Papers,3.0,0.0
2,SAM,NaN,Defaulted,Mossack Fonseca,HKG,NaN,The Panama Papers data is current through 2015,Panama Papers,1.0,1.0
3,SAM,NaN,Defaulted,Mossack Fonseca,HKG,NaN,The Panama Papers data is current through 2015,Panama Papers,1.0,4.0
4,SAM,NaN,Changed agent,Mossack Fonseca,HKG,NaN,The Panama Papers data is current through 2015,Panama Papers,4.0,2.0
